## Making a Lightcurve from PTF catalog data

In [20]:
import numpy as np
import astropy.io.fits as fits
import astropy.coordinates as coords
import astropy.units as u
from astropy.time import Time
from glob import glob

In [8]:
reference_catalog = '../data/PTF_Refims_Files/PTF_d022683_f02_c06_u000114210_p12_sexcat.ctlg'
# select R-band data (f02)
epoch_catalogs = glob('../data/PTF_Procim_Files/PTF*f02*.ctlg')

In [10]:
len(epoch_catalogs)

72

In [91]:
def load_ref_catalog(reference_catalog):
    hdus = fits.open(reference_catalog)
    data = hdus[1].data
    # filter flagged detections
    w = (data['flags'] & 506 == 0)
    data = data[w]

    ref_coords = coords.SkyCoord(data['X_WORLD'], data['Y_WORLD'],frame='icrs',unit='deg')

    return np.vstack([data['MAG_AUTO'],data['MAGERR_AUTO']]).T, ref_coords

In [92]:
ref_mags, ref_coords = load_ref_catalog(reference_catalog)

In [94]:
len(ref_mags)

14043

In [137]:
# optional: progress bar
def load_progress_bar():
    import uuid
    from IPython.display import HTML, Javascript, display
    import time

    divid = str(uuid.uuid4())
    pb = HTML(
"""
<div style="border: 1px solid black; width:500px">
  <div id="%s" style="background-color:blue; width:0%%">&nbsp;</div>
</div> 
""" % divid)
    display(pb)
    return divid
#for i in range(1,101):
#    time.sleep(0.1)
    
#    display(Javascript("$('div#%s').width('%i%%')" % (divid, i)))

In [138]:
def crossmatch_epochs(reference_coords, epoch_catalogs):
    
    n_stars = len(reference_coords)
    n_epochs = len(epoch_catalogs)
    
    mags = np.ma.zeros([n_stars, n_epochs])
    magerrs = np.ma.zeros([n_stars, n_epochs])
    mjds = np.ma.zeros(n_epochs)
    
    divid = load_progress_bar()
    for i, catalog in enumerate(epoch_catalogs):
        #print catalog
        hdus = fits.open(catalog)
        data = hdus[1].data
        hdr = hdus[2].header
        # filter flagged detections
        w = ((data['flags'] & 506 == 0) & (data['imaflags_iso'] & 1821 == 0))
        data = data[w]

        epoch_coords = coords.SkyCoord(data['X_WORLD'], data['Y_WORLD'],frame='icrs',unit='deg')
        idx, sep, dist = coords.match_coordinates_sky(epoch_coords, reference_coords)
        
        wmatch = (sep <= 1.5*u.arcsec)
        
        # store data
        if np.sum(wmatch):
            mags[idx[wmatch],i] = data[wmatch]['MAG_AUTO'] + data[wmatch]['ZEROPOINT']
            magerrs[idx[wmatch],i] = data[wmatch]['MAGERR_AUTO']
            mjds[i] = hdr['OBSMJD']

        display(Javascript("$('div#%s').width('%i%%')" % (divid, i//n_epochs)))
    return mjds, mags, magerrs

In [139]:
mjds,mags,magerrs = crossmatch_epochs(ref_coords, epoch_catalogs)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [128]:
mags.max()

93.9208984375